<p style="font-weight:bold;"> <span style="font-size: 36px"> Initialize Data </span> </p>

In [ ]:
#!import "../ifrs17/Import/Importers"
//#!import "//ifrs17/dev/Import/Importers" 

In [ ]:
#!import "../ifrs17/CalculationEngine"

---

    TODO: this is a trick, remove later when import format is ready and implemented in the calculation engine

In [ ]:
public static IImportVariable GetImport(this IImportVariable import) => import; 

In [ ]:
var importbck = Import.GetImport();

In [ ]:
var Import = importbck;

---

In [ ]:
// not working because enums are not supported (also this requires the excel cells format to be tuned adequately)
//await Domain.GenerateFromFile("Dimensions.xlsx").ExecuteAsync()

In [ ]:
/*
var domain = Domain.CreateDomain("Dimension")
                .WithType<Novelty>()            .WithType<AocType>()                    .WithType<PnlVariableType>()            .WithType<AocConfiguration>()
                .WithType<PvAmountType>()       .WithType<DeferrableAmountType>()       .WithType<CalculationType>()            .WithType<RiskDriver>()
                .WithType<EconomicBasis>()      .WithType<EstimateType>()               .WithType<ValuationApproach>()          .WithType<LineOfBusiness>()
                .WithType<OciType>()            .WithType<LiabilityType>()              .WithType<Profitability>()              .WithType<Currency>()
                .WithType<ReportingNode>()      .WithType<Partner>()                    .WithType<CreditRiskRating>()           .WithType<Scenario>()
                .WithType<ProjectionConfiguration>()    
                .WithType<PartitionByReportingNode>()            // TODO: automatize partitions
                .WithType<PartitionByReportingNodeAndPeriod>()   // TODO: not working cause the name is too long for excel
                .ToDomain();
*/

In [ ]:
// not usable at the moment for the reason outlined above
//await Import.FromFile("Dimensions.xlsx").WithDomain(domain).WithTarget(DataSource).ExecuteAsync()

In [ ]:
// FEEDBACK: use domain (e.g. IFRS17.Import), see above
await Import.FromFile("Dimensions.xlsx")
            .WithType<Novelty>()                  .WithType<AocType>()                   .WithType<PnlVariableType>()            .WithType<AocConfiguration>()
            .WithType<PvAmountType>()             .WithType<DeferrableAmountType>()      //.WithType<CalculationType>()            
            .WithType<RiskDriver>()
            .WithType<EconomicBasis>()            .WithType<EstimateType>()              .WithType<ValuationApproach>()          .WithType<LineOfBusiness>()
            .WithType<OciType>()                  .WithType<LiabilityType>()             .WithType<Profitability>()              .WithType<Currency>()
            .WithType<ReportingNode>()            .WithType<Partner>()                   .WithType<CreditRiskRating>()           .WithType<Scenario>()
            .WithType<ProjectionConfiguration>()
            .WithType<PartitionByReportingNode>()                                                            // TODO: automatize partitions
            .WithType<PartitionByReportingNodeAndPeriod>("PartitionByReportingNodeAndPeri")                  // TODO: ugly
            .WithTarget(DataSource)
            .ExecuteAsync()

In [ ]:
await UploadDataNodesAsync("DataNodes.xlsx")

In [ ]:
// TODO: set the imported DataNodes to Active by default
//await UploadDataNodeStateAsync("DataNodeStates.csv")

In [ ]:
// TODO: 1. map template not working, 2. set default values
//await UploadDataNodeParameterAsync("DataNodeParameters.csv")

# TMP import api

In [ ]:
// TODO: temporary workaround while waiting for the new unified import api
public class MyImport 
{
    public Systemorph.Vertex.Import.IImportVariable Import {get; init;}
    public Systemorph.Vertex.DataSource.IDataSourceVariable DataSource {get; private set;}
    public IWorkspace Workspace {get; init;}
    
    private Func<string, Task<ActivityLog>> ImportCashflowAsync {get; init;}
    private Func<string, Task<ActivityLog>> ImportActualAsync {get; init;}
    private Func<string, Task<ActivityLog>> ImportDataNodeAsync {get; init;}
    
    private string fileName;
    private string format; 
    
    public MyImport (IImportVariable import,
                     IWorkspace workspace, 
                     Func<string, Task<ActivityLog>> importDataNodeAsync,
                     Func<string, Task<ActivityLog>> importCashflowAsync, 
                     Func<string, Task<ActivityLog>> importActualAsync) 
    {
        Import = import;
        Workspace = workspace;
        ImportDataNodeAsync = importDataNodeAsync;
        ImportCashflowAsync = importCashflowAsync;
        ImportActualAsync = importActualAsync;
    }
    
    public IImportVariable GetImport() => Import;
    
    public MyImport FromFile (string fn)    { fileName = fn; return this; }
    public MyImport WithFormat (string f)   { format = f; return this; }
    public MyImport WithType<T> ()          { format = typeof(T).Name; return this; }
    public MyImport WithTarget (Systemorph.Vertex.DataSource.IDataSourceVariable datasource) { DataSource = datasource; return this; }
    
    public async Task<Systemorph.Vertex.Activities.ActivityLog[]> ExecuteAsync () {
        Systemorph.Vertex.Activities.ActivityLog[] logs;
        switch(format) 
        {
            case("DataNodes") : logs = (await ImportDataNodeAsync(fileName)).RepeatOnce().ToArray(); break;

            // TODO: add property to stress that this is yearly
            case("YieldCurve") : logs = (await Import.FromFile(fileName).WithType<YieldCurve>().WithTarget(DataSource).ExecuteAsync()).RepeatOnce().ToArray();  break;
                
            // TODO: use default for Calculation Type (BE, CU, RA) and Novelty
            // Objection: is is quite fundamental to state which calculation type the cf is, better to delegate this to the user
            // Possibility: fuse calculation type and amount type?
            case("Cashflows") : logs = (await ImportCashflowAsync(fileName)).RepeatOnce().ToArray();  break;

            case("Actuals") : logs = (await ImportActualAsync(fileName)).RepeatOnce().ToArray();  break;

            default : logs = (new Systemorph.Vertex.Activities.ActivityLog(new Guid(), new DateTime(), new DateTime(), "", ActivityLogStatus.Succeeded, new(), new(), new() )).RepeatOnce().ToArray();  break;
        };
        // TODO: clarify how this works, e.g. whether the data is cached or not
        // TODO: use DataSOurce in the report module so that this line is not needed
        Workspace.InitializeFrom(DataSource);
        return logs;
    }
}

In [ ]:
var Import = new MyImport(importbck, Workspace, UploadDataNodesAsync, UploadCashflowsAsync, UploadActualAsync);

In [ ]:
await Import.FromFile("DataNodes.xlsx").WithFormat("DataNodes").WithTarget(DataSource).ExecuteAsync()